# Steady state solutions

- DiffEq docs: https://docs.sciml.ai/DiffEqDocs/stable/solvers/steady_state_solve/
- ModelingToolkit docs: https://docs.sciml.ai/ModelingToolkit/stable/tutorials/nonlinear/
- NonlinearSolve docs: https://docs.sciml.ai/NonlinearSolve/stable/

Solving steady state solutions for an ODE system is to find a combination of state variables such that their derivatives are all zeroes. Their are two ways:

- Running the ODE solver until a steady state is reached (`DynamicSS()` and `DifferentialEquations.jl`)
- Using a root-finding algorithm to find a steady state (`SSRootfind()` and `NonlinearSolve.jl`)

## Defining a steady state problem

In [ ]:
using DifferentialEquations

model(u, p, t) = 1 - p * u

p = 1.0 ## Parameter
u0 = 0.0 ## Initial condition
prob = SteadyStateProblem(model, u0, p) ## Define the problem
alg = DynamicSS(Tsit5())
sol = solve(prob, alg) ## Solve the problem

The result should be close to 1.0.

## Modelingtoolkit solving nonlinear systems 

Use `NonlinearSolve.jl` and `NonlinearSystem()`.

In [ ]:
using ModelingToolkit
using NonlinearSolve

@variables x y z
@parameters σ ρ β

eqs = [
    0 ~ σ * (y-x),
    0 ~ x * (ρ - z) - y,
    0 ~ x * y - β * z
]

In [ ]:
@named ns = NonlinearSystem(eqs, [x, y, z], [σ, ρ, β])

In [ ]:
guess = [x => 1.0, y => 0.0, z => 0.0]
ps = [σ => 10.0, ρ => 26.0, β => 8/3]
prob = NonlinearProblem(ns, guess, ps)
sol = solve(prob, NewtonRaphson()) ## The results should be all zeroes

Another nonlinear system example with `structural_simplify()`.

In [ ]:
@parameters t
@variables u1(t) u2(t) u3(t) u4(t) u5(t)

eqs = [
    0 ~ u1 - sin(u5)
    0 ~ u2 - cos(u1)
    0 ~ u3 - hypot(u1, u2)
    0 ~ u4 - hypot(u2, u3)
    0 ~ u5 - hypot(u4, u1)
]

@named sys = NonlinearSystem(eqs, [u1, u2, u3, u4, u5], [])

You can simplify the problem using `structural_simplify()`.

There will be only one state variable left. The solve can solve the problem faster.

In [ ]:
simple_sys = structural_simplify(sys)

prob = NonlinearProblem(simple_sys, [u5=>0.0])
sol = solve(prob, NewtonRaphson())

The answer should be 1.6 and 1.0.

In [ ]:
@show sol[u5] sol[u1];  ## 1.6 and 1.0